In [5]:
	
import tensorflow as tf
from alexnet_model import AlexNet

def train_alexnet():
    # Data Generators (Normalization Only)
    train_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    valid_test_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

    
    # Loading Training, Validation, and Test Data
    train_generator = train_data.flow_from_directory(
        '/Users/maximilianstumpf/Desktop/UCLA/Math156 - Machine Learning/Project/Data/Processed/train',
        batch_size=64,
        class_mode='categorical',
        shuffle=True
    )
    validation_generator = valid_test_data.flow_from_directory(
        '/Users/maximilianstumpf/Desktop/UCLA/Math156 - Machine Learning/Project/Data/Processed/val',
        batch_size=64,
        class_mode='categorical',
        shuffle=True
    )
    test_generator = valid_test_data.flow_from_directory(
        '/Users/maximilianstumpf/Desktop/UCLA/Math156 - Machine Learning/Project/Data/Processed/test',
        batch_size=64,
        class_mode='categorical',
        shuffle=False
    )
    
    # Define AlexNet Model
    num_classes = 10  # Adjust to your dataset's number of classes
    model = AlexNet(num_classes=num_classes)
    
    # Learning Rate Scheduler
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate = 0.01,
        decay_steps = 10000,
        alpha = 0.0001
    )
    
    
    # Optimizer
    optimizer = tf.keras.optimizers.SGD(
        learning_rate=lr_schedule,
        momentum=0.9,
        nesterov=True
    )
    
    # Compile the Model
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Training the Model
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        callbacks=[
            tf.keras.callbacks.ModelCheckpoint(
                'alexnet_checkpoint.keras',  # Save the best model checkpoint
                save_best_only=True,
                monitor='val_accuracy'
            ),
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=5,               # Stop training if val_loss doesn't improve for 5 epochs
                restore_best_weights=True
            )
        ]
    )
    
    # Save Final Weights
    model.save_weights('alexnet_final.weights.h5')
    
    # Evaluate on Test Data
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    return model, history

# Run the Training
if __name__ == "__main__":
    model, history = train_alexnet()

Found 43954 images belonging to 10 classes.
Found 14651 images belonging to 10 classes.
Found 14652 images belonging to 10 classes.
Epoch 1/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 344s 499ms/step - accuracy: 0.1817 - loss: 2.2496 - val_accuracy: 0.1903 - val_loss: 2.2356
Epoch 2/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 341s 496ms/step - accuracy: 0.2505 - loss: 2.0944 - val_accuracy: 0.6813 - val_loss: 1.0611
Epoch 3/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 351s 510ms/step - accuracy: 0.6855 - loss: 1.0168 - val_accuracy: 0.7498 - val_loss: 0.8104
Epoch 4/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 340s 494ms/step - accuracy: 0.7842 - loss: 0.6928 - val_accuracy: 0.8118 - val_loss: 0.6171
Epoch 5/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 317s 462ms/step - accuracy: 0.8321 - loss: 0.5501 - val_accuracy: 0.8283 - val_loss: 0.5669
Epoch 6/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 327s 476ms/step - accuracy: 0.8565 - loss: 0.4727 - val_accuracy: 0.8359 - val_loss: 0.5466
Epoch 7/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 331s 481ms/step - accuracy: 0.8732 -